In [4]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
import time

from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from typing import Optional
from typing import List


from langchain.callbacks import get_openai_callback



# Carregar variáveis de ambiente
dotenv_path = find_dotenv()
_ = load_dotenv(dotenv_path)

class ResumoCurriculo(BaseModel):
    nome_completo: Optional[str] = Field(None, description="Nome completo do candidato. Deve estar escrito todo em letra maiúscula.")
    experiencia: List[str] = Field(None, description="Resumo da experiência profissional")
    habilidades: List[str] = Field(None, description="Principais habilidades")
    educacao: List[str] = Field(None, description="Formação educacional")
    idiomas: List[str] = Field(None, description="Idiomas conhecidos")

    def dict(self, *args, **kwargs):
        return super().dict(*args, **kwargs, exclude_none=True)

In [5]:
def iniciar_modelo():
    model = ChatOpenAI(model='gpt-4o-mini')
    return model

In [6]:
texto_markdown = '''## Armando Gomes

## ADMINISTRAÇÃO

<!-- image -->

Telefone: (12) 3456 - 7890 ola@grandesite.com.br

Rua Alegre, 123 - Cidade Brasileira, Estado, País

<!-- image -->

## OBJETIVO

Ter a oportunidade de desenvolver - me profissionalmente, colocando em prática aspectos abordados em sala de aula. Fazer um bom trabalho em equipe, colaborando assim com meu crescimento profissional e com a instituição a qual fui engajada.

<!-- image -->

## FORMAÇÃO

- Escola Faustino Técnico em Informática, integrado ao ensino médio. Integral. Início em 03/ 2012 Conclusão em 03/2016
- Faculdade Borcelle
- Técnico em Administração. Vespertino. Início em 08/2018 Conclusão em 08/2022

<!-- image -->

## EXPERIÊNCIAS

- 25/03/2015 - 25/03/2016: Empresa Ícaro Área de atuação: Auxiliar Financeiro/ Administrativo Atividade: Estágio Duração: 1 ano
- 23/10/2017 - 31/03/2019 : Queirós e Parceiros Área de atuação: Auxiliar Financeiro Atividade: Funcionário Duração: 4 meses

<!-- image -->

## HABILIDADES

- Boa Comunicação
- Trabalho em equipe
- Proativo
- Resiliência

<!-- image -->

<!-- image -->

<!-- image -->'''

start_time = time.time()

parser = PydanticOutputParser(pydantic_object=ResumoCurriculo)
format_instructions = parser.get_format_instructions()

system_prompt = '''
    Você é um assistente de RH com muita experiência na análise crítica de currículos.
    A seguir você será apresentado ao curriculo e terá instruções precisas para parsea-lo.
    Seu objetivo é extrair as informações do curriculo e retornar um resumo do mesmo em formato JSON.
    Lembre-se de que esses curriculos estão em formato markdown e foram extraídos de arquivos pdf por um certo processo de extração de texto.
    Portanto, as vezes, as informações não estarão perfeitamente em seu capítulos correspondentes. Assim, analise as informação como um todo.
    Se você não encontrar alguma informação pedida no corriculo coloque "Não encontrado" no campo correspondente.
    Necessariamente preciso que o output seja em formato JSON para análises posteriores.
    Ignore strings no formato "<!-- image -->", elas representam posições onde imagens foram detectadas no curriculo.
    Não escreva nada além do formato JSON solicitado.
'''

human_prompt = "{format_instructions}\n\n Currículo do candidato para resumir:\n\n{curriculo}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt)
]).partial(format_instructions=format_instructions)

modelo = iniciar_modelo()

chain = prompt | modelo | parser



In [ ]:
with get_openai_callback() as cb:
    resultado = chain.invoke({"curriculo": texto_markdown})

    processing_time = time.time() - start_time

    tokens_entrada = cb.prompt_tokens
    tokens_saida = cb.completion_tokens
    custo_chamada = cb.total_cost
    tempo_processamento = processing_time

    print("Tokens de entrada:", tokens_entrada)
    print("Tokens de saída:", tokens_saida)
    print("Total de tokens:", cb.total_tokens)
    print("Custo da chamada (USD):", custo_chamada)
    print("Tempo de processamento (s):", tempo_processamento)

In [56]:
print(type(resultado.model_dump_json()))
resultado.model_dump_json()

<class 'str'>


'{"nome_completo":"ARMANDO GOMES","experiencia":["25/03/2015 - 25/03/2016: Empresa Ícaro - Auxiliar Financeiro/ Administrativo - Estágio - Duração: 1 ano","23/10/2017 - 31/03/2019: Queirós e Parceiros - Auxiliar Financeiro - Funcionário - Duração: 4 meses"],"habilidades":["Boa Comunicação","Trabalho em equipe","Proativo","Resiliência"],"educacao":["Escola Faustino - Técnico em Informática, integrado ao ensino médio - Início em 03/2012 - Conclusão em 03/2016","Faculdade Borcelle - Técnico em Administração - Início em 08/2018 - Conclusão em 08/2022"],"idiomas":["Não encontrado"]}'

In [7]:
modelo = iniciar_modelo()

In [9]:
modelo.model_name

'gpt-4o-mini'

In [23]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    resultado = chain.invoke({"curriculo": texto_markdown})

    processing_time = time.time() - start_time

    print("Tokens de entrada:", cb.prompt_tokens)
    print("Tokens de saída:", cb.completion_tokens)
    print("Total de tokens:", cb.total_tokens)
    print("Custo da chamada (USD):", cb.total_cost)
    print("Tempo de processamento (s):", processing_time)

Tokens de entrada: 556
Tokens de saída: 56
Total de tokens: 612
Custo da chamada (USD): 0.000117
Tempo de processamento (s): 505.9554226398468


In [24]:
print("Tokens de entrada:", cb.prompt_tokens)
print("Tokens de saída:", cb.completion_tokens)
print("Total de tokens:", cb.total_tokens)
print("Custo da chamada (USD):", cb.total_cost)
print("Tempo de processamento (s):", processing_time)

Tokens de entrada: 556
Tokens de saída: 56
Total de tokens: 612
Custo da chamada (USD): 0.000117
Tempo de processamento (s): 505.9554226398468
